In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import cftime._cftime as cft
import matplotlib.gridspec as gridspec
import nctoolkit as nc

plt.rcParams['axes.facecolor']='black'
#plt.style.use('dark_background')
plt.style.use('default')

plt.rcParams.update({'font.size': 20})

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values
    try:
        xrdataset = xrdataset.rename_dims({'x_grid_T': 'x' ,'y_grid_T': 'y'})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3t.cumsum('deptht')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3u.cumsum('depthu')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3v.cumsum('depthv')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3w.cumsum('depthw')})
    except:
        pass
    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
def combine_coords(datasets):
    zcoords  = []
    for ds in datasets:
        zcoords.append(ds.z.mean('time_counter'))
    zcoord   = sum(zcoords) / len(zcoords)
    assigned = []
    for ds in datasets:
        assigned.append(ds.assign_coords({'z0': zcoord}))
    return tuple(assigned)

In [ ]:
def create_xgcm_domain(mesh, domain):
    ds = xr.merge([mesh, domain]).isel(time_counter=0)
    coords = dict(
        x_c = ds.glamt.isel(y=5).values,
        x_f = ds.glamu.isel(y=5).values,
        y_c = ds.gphit.isel(x=5).values,
        y_f = ds.gphiv.isel(x=5).values,
        z_c = ds.gdept_1d.values,
        z_f = ds.gdepw_1d.values
    )
    data_vars = dict(
        e1t = (['y_c','x_c'], ds.e1t.values),
        e1u = (['y_c','x_f'], ds.e1u.values),
        e1v = (['y_f','x_c'], ds.e1v.values),
        e1f = (['y_f','x_f'], ds.e1f.values),
        e2t = (['y_c','x_c'], ds.e2t.values),
        e2u = (['y_c','x_f'], ds.e2u.values),
        e2v = (['y_f','x_c'], ds.e2v.values),
        e2f = (['y_f','x_f'], ds.e2f.values),
        e3t = (['z_c','y_c','x_c'], ds.e3t_0.values),
        e3u = (['z_c','y_c','x_f'], ds.e3u_0.values),
        e3v = (['z_c','y_f','x_c'], ds.e3v_0.values),
        e3f = (['z_c','y_f','x_f'], ds.e3f_0.values)
    )
    return xr.Dataset(data_vars=data_vars, coords=coords)

In [ ]:
def V_transport_on_z(domain, ds):
    ds_new = domain.copy()
    #vtra = (ds.voce * ds.e3v * ds.e1v).values
    vtra = ds.vocetr_eff.values
    ds_new['vtra']      = (['time_counter','z_f', 'y_f', 'x_c'], vtra)
    ds_new['depthw'] = (['time_counter','z_f', 'y_f', 'x_c'], - ds.z.values)
    ds_new = ds_new[['vtra','depthw']]
    ds_new = ds_new.assign_coords({'z_c':domain.z_c.values[:-1]})
    grid = xgcm.Grid(ds_new,
        coords={
            "X": {"center": "x_c"},
            "Y": {"right": "y_f"},
            "Z": {"center": "z_c", "outer": "z_f"}
        },
        periodic=False)
    z_target = domain.z_c.values      
    
    vtra_transformed = grid.transform(ds_new.vtra.isel(z_f=slice(0,-1)),
        'Z',
        z_target,
        method='conservative',
        target_data=ds_new.depthw)
    vtra_transformed = vtra_transformed.assign_coords(z=('depthw', - vtra_transformed.depthw.values))
    return vtra_transformed

In [ ]:
import numpy as np
def V_transport_on_sigma(domain, ds, sigma0):
    ds_new = domain.isel(z_c=slice(0,-1)).copy()
    #vtra = (ds.voce * ds.e3v * ds.e1v).values

    ds_new['vtra']      = (['time_counter','z_c', 'y_f', 'x_c'], ds.vocetr_eff.isel(depthv=slice(0,-1)).data)
    ds_new['sigma0'] = (['time_counter','z_c', 'y_c', 'x_c'], sigma0.isel(deptht=slice(0,-1)).data)
    ds_new = ds_new[['vtra','sigma0', 'e3v']]
    ds_new = ds_new.assign_coords({'z_f':domain.z_f.values})
    grid = xgcm.Grid(ds_new,
        coords={
            "X": {"center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"center": "z_c", "outer": "z_f"}
        },
        periodic=False)
    ds_new['sigma_v'] = grid.interp(ds_new.sigma0, 'Y')
    ds_new['sigma_outer'] = grid.interp(ds_new.sigma_v, 'Z',  boundary='extend')
    #sigma_1d = ds_new.sigma_outer.isel(time_counter=-2, x_c=slice(2,-2)).mean(['x_c', 'y_f'])
    sigma_target = np.array([24, 24.2, 24.4, 24.65, 24.9, 25.15, 25.4, 25.65,
                          25.9, 26.15, 26.4, 26.57, 26.75, 26.90, 27.05, 27.17,
                          27.30, 27.37, 27.45, 27.52, 27.58,
                          27.63, 27.68, 27.72, 27.75, 27.78, 27.80, 27.815,
                          27.83, 27.845, 27.86, 27.875, 27.89,
                          27.905, 27.92, 27.935, 27.95, 27.965,
                          27.98, 27.99, 28.01, 28.025,
                          28.04, 28.055, 28.07, 28.085, 28.1, 28.35,
                          28.6, 28.8, 29.1])
    # e3v_target = ds_new.e3v.isel(y_f=100, x_c=40)      
    
    vtra_transformed = grid.transform(ds_new.vtra,
        'Z',
        sigma_target,
        method='conservative',
        target_data=ds_new.sigma_outer)
    
    e3v_transformed = grid.transform(ds_new.e3v,
        'Z',
        sigma_target,
        method='conservative',
        target_data=ds_new.sigma_outer.isel(time_counter=-2))
    # vtra_transformed = vtra_transformed.assign_coords(z=('depthw', - vtra_transformed.depthw.values))
    return vtra_transformed, e3v_transformed

In [ ]:
def V_sigma_on_z(domain, da):
    ds_new = domain.isel(z_c=slice(0,-1)).copy()
    
    #vtra = (ds.voce * ds.e3v * ds.e1v).values

    ds_new['vtra']      = (['time_counter','z_c', 'y_f', 'x_c'], ds.vocetr_eff.isel(depthv=slice(0,-1)).data)
    ds_new['sigma0'] = (['time_counter','z_c', 'y_c', 'x_c'], sigma0.isel(deptht=slice(0,-1)).data)
    ds_new = ds_new[['vtra','sigma0']]
    ds_new = ds_new.assign_coords({'z_f':domain.z_f.values})
    grid = xgcm.Grid(ds_new,
        coords={
            "X": {"center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"center": "z_c", "outer": "z_f"}
        },
        periodic=False)
    ds_new['sigma_v'] = grid.interp(ds_new.sigma0, 'Y')
    ds_new['sigma_outer'] = grid.interp(ds_new.sigma_v, 'Z',  boundary='extend')
    sigma_1d = ds_new.sigma_outer.isel(time_counter=-2, x_c=slice(2,-2)).mean(['x_c', 'y_f'])
    
    z_target = sigma_1d.values      
    
    vtra_transformed = grid.transform(ds_new.vtra,
        'Z',
        z_target,
        method='conservative',
        target_data=ds_new.sigma_outer)
    
    # vtra_transformed = vtra_transformed.assign_coords(z=('depthw', - vtra_transformed.depthw.values))
    return vtra_transformed

In [ ]:
def combine_restarts(path:str, variable:str, grid:str) -> xr.DataArray:
    """
    Combine the data in all restart files to yield one variable as DataArray
    """
    

In [ ]:
path   = "/data/dkamm/nemo_output/NEVERWORLD/"
#path_scratch = "/gpfsscratch/rech/omr/uym68qx/"

In [ ]:
U_new   = clean_coords(xr.load_dataset(path + 'GURVAN_5y/NEVERWORLD_grid_U.nc'), 2000)
U_old   = clean_coords(xr.load_dataset(path + 'OLD_5y/NEVERWORLD_grid_U.nc'), 2000)
U_triad = clean_coords(xr.load_dataset(path + 'TRIAD_5y/NEVERWORLD_grid_U.nc'), 2000)

In [ ]:
U_new, U_old, U_triad = combine_coords([U_new, U_old, U_triad])

In [ ]:
T_new   = clean_coords(xr.load_dataset(path + 'GURVAN_5y/NEVERWORLD_grid_T.nc'), 2000)
T_old   = clean_coords(xr.load_dataset(path + 'OLD_5y/NEVERWORLD_grid_T.nc'), 2000)
T_triad = clean_coords(xr.load_dataset(path + 'TRIAD_5y/NEVERWORLD_grid_T.nc'), 2000)

In [ ]:
T_new, T_old, T_triad = combine_coords([T_new, T_old, T_triad])

In [ ]:
V_new   = clean_coords(xr.load_dataset(path + 'GURVAN_5y/NEVERWORLD_grid_V.nc'), 2000)
V_old   = clean_coords(xr.load_dataset(path + 'OLD_5y/NEVERWORLD_grid_V.nc'), 2000)
V_triad = clean_coords(xr.load_dataset(path + 'TRIAD_5y/NEVERWORLD_grid_V.nc'), 2000)

In [ ]:
V_new, V_old, V_triad = combine_coords([V_new, V_old, V_triad])

# Top zonal velocities at 30° N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)
col1 = U_old.uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[0,0], add_colorbar=False, cmap='RdBu_r')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[m/s]')
U_new.uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
U_triad.uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[2,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
cmap2 = (U_new - U_old).uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[0,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(cmap2, ax=axs[:,1], location='right', label='[m/s]')
(U_new - U_triad).uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(U_old - U_triad).uoce.isel(y=120, time_counter=-2, depthu=slice(0,25)).plot(y='z0', ax=axs[2,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
axs[0,0].set_title('Old', fontsize=15)
axs[1,0].set_title('New', fontsize=15)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)

# Bottom zonal velocities at 30° N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)
col1 = U_old.uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[0,0], add_colorbar=False, cmap='RdBu_r')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[m/s]')
U_new.uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
U_triad.uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[2,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
cmap2 = (U_new - U_old).uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[0,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(cmap2, ax=axs[:,1], location='right', label='[m/s]')
(U_new - U_triad).uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(U_old - U_triad).uoce.isel(y=120, time_counter=-2, depthu=slice(25, -1)).plot(y='z0', ax=axs[2,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
axs[0,0].set_title('Old', fontsize=15)
axs[1,0].set_title('New', fontsize=15)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)

# Top temperature  at 30°N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)

col1  = T_triad.toce.isel(y=120, time_counter=-2, deptht=slice(0,25), x=slice(1,-1)).plot(y='z0', ax=axs[2,0], add_colorbar=False, cmap='plasma')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[°C]')
T_old.toce.isel(y=120, time_counter=-2, deptht=slice(0,25), x=slice(1,-1)).plot(y='z0', ax=axs[0,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='plasma')
T_new.toce.isel(y=120, time_counter=-2, deptht=slice(0,25), x=slice(1,-1)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='plasma')

col2 = (T_old - T_triad).toce.isel(y=120, time_counter=-2, deptht=slice(0,25)).plot(y='z0', ax=axs[2,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(col2, ax=axs[:,1], location='right', label='[°C]')
(T_new - T_old).toce.isel(y=120, time_counter=-2, deptht=slice(0,25)).plot(y='z0', ax=axs[0,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(T_new - T_triad).toce.isel(y=120, time_counter=-2, deptht=slice(0,25)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')

axs[0,0].set_title('Old', fontsize=20)
axs[1,0].set_title('New', fontsize=20)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)



# Bottom temperature  at 30°N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)

col1  = T_triad.toce.isel(y=120, time_counter=-2, deptht=slice(25,-1), x=slice(1,-1)).plot(y='z0', ax=axs[2,0], add_colorbar=False, cmap='plasma')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[°C]')
T_old.toce.isel(y=120, time_counter=-2, deptht=slice(25,-1), x=slice(1,-1)).plot(y='z0', ax=axs[0,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='plasma')
T_new.toce.isel(y=120, time_counter=-2, deptht=slice(25,-1), x=slice(1,-1)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='plasma')

col2 = (T_old - T_triad).toce.isel(y=120, time_counter=-2, deptht=slice(25,-1)).plot(y='z0', ax=axs[2,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(col2, ax=axs[:,1], location='right', label='[°C]')
(T_new - T_old).toce.isel(y=120, time_counter=-2, deptht=slice(25,-1)).plot(y='z0', ax=axs[0,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(T_new - T_triad).toce.isel(y=120, time_counter=-2, deptht=slice(25,-1)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')

axs[0,0].set_title('Old', fontsize=20)
axs[1,0].set_title('New', fontsize=20)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)



# Top meridional velocities at 30° N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)
col1 = V_old.voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[0,0], add_colorbar=False, cmap='RdBu_r')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[m/s]')
V_new.voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
V_triad.voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[2,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
cmap2 = (V_new - V_old).voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[0,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(cmap2, ax=axs[:,1], location='right', label='[m/s]')
(V_new - V_triad).voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(V_old - V_triad).voce.isel(y=120, time_counter=-2, depthv=slice(0,25)).plot(y='z0', ax=axs[2,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
axs[0,0].set_title('Old', fontsize=15)
axs[1,0].set_title('New', fontsize=15)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)

# Bottom meridional velocities at 30° N after five years of integration

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)
col1 = V_old.voce.isel(y=120, time_counter=-2, depthv=slice(25, -1)).plot(y='z0', ax=axs[0,0], add_colorbar=False, cmap='RdBu_r')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[m/s]')
V_new.voce.isel(y=120, time_counter=-2, depthv=slice(25,-1)).plot(y='z0', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
V_triad.voce.isel(y=120, time_counter=-2, depthv=slice(25,-1)).plot(y='z0', ax=axs[2,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
cmap2 = (V_new - V_old).voce.isel(y=120, time_counter=-2, depthv=slice(25,-1)).plot(y='z0', ax=axs[0,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(cmap2, ax=axs[:,1], location='right', label='[m/s]')
(V_new - V_triad).voce.isel(y=120, time_counter=-2, depthv=slice(25,-1)).plot(y='z0', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(V_old - V_triad).voce.isel(y=120, time_counter=-2, depthv=slice(25,-1)).plot(y='z0', ax=axs[2,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
axs[0,0].set_title('Old', fontsize=15)
axs[1,0].set_title('New', fontsize=15)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)

In [ ]:
import cartopy.crs as ccrs

#U_s = clean_coords(xr.load_dataset(path + 'Scottia_low_150y/restart0/NEVERWORLD_grid_U.nc'), 1700)
U_s = combine_coords([U_s])

In [ ]:
U_s[0]

In [ ]:
U_plt = U_s[0].assign_coords({'x': U_s[0].x.values - 60.}).z0.isel(depthu=-1)

In [ ]:
U_plt_np = np.concatenate((np.zeros((1,52)),np.concatenate((np.zeros((199,1)), U_plt.values, np.zeros((199,1))), axis=1), np.zeros((1,52))), axis=0)
U_plt_np[25:56,-1] = U_plt_np[25:56,-2]
U_plt_np[25:56,0]  = U_plt_np[25:56,1]

In [ ]:
U_plot = xr.DataArray(U_plt_np,
    coords={
        'y':np.concatenate(([-70.2], U_plt.y.values, [70.2])),
        'x':np.concatenate(([-61.], U_plt.x.values, [-10.]))
    }
)

In [ ]:
p = U_plot.plot(

    subplot_kws=dict(projection=ccrs.Orthographic(-30., 0.), facecolor="black"),

    transform=ccrs.PlateCarree(),

    figsize=(20,20),

    add_colorbar=False,
    # cmap='inferno'
)
p.axes.set_global()

p.axes.coastlines(color='white', linewidth=3)
plt.title('')
plt.colorbar(p,fraction=0.046, pad=0.04)


In [ ]:
 # U_new = U_new.assign_coords({'x': U_new.x.values - 60.})

In [ ]:
test = np.mgrid[1:37, 1:50][0]

In [ ]:
test[0].shape

In [ ]:
(- U_s.e3u.cumsum('depthu').values).shape

In [ ]:
import numpy as np

In [ ]:
np.mgrid[1:37, 0:50, 0:199,  0:50][0]

In [ ]:
U_plt['vertical level'] = (['depthu', 'x', 'y', 'time_counter'], np.mgrid[1:37, 0:50, 0:199,  0:50][0])

In [ ]:
(- U_plt.e3u.isel(time_counter=-2, x=-1).cumsum('depthu')).plot.line(x='y', add_legend=False, color='white', figsize=(20,10))
plt.title('')
#plt.xlim(-60,-11)
plt.hlines(-1000, xmin=-60, xmax=-11, color='darkred', linewidth=6)
plt.ylabel('depth')

In [ ]:
U_s['vertical level'].isel(y=120, time_counter=-2).plot.contourf(y='z0', figsize=(20,10), levels=36, cmap='viridis')
(- U_s.e3u.isel(time_counter=-2, y=120).cumsum('depthu')).plot.line(x='x', add_legend=False, color='black', linestyle='-')
plt.step(U_s.x.values, -U_s.steps.values, where='mid', color='white', label='ORCA1')
plt.title('')
plt.hlines(-1000, xmin=-60, xmax=-11, color='darkred', linewidth=6)
plt.ylabel('depth [m]')
plt.xlabel('longitude [°E]')
plt.legend(labelcolor='white')

In [ ]:
U_s['vertical level'].isel(y=120, time_counter=-2).shape

In [ ]:
T_new.toce.isel(time_counter=-2, x=10, deptht=slice(0,20)).plot.contourf(y='z0',vmin=10, figsize=(20,10))

# Interpolation

In [ ]:
#xr.load_dataset(path + 'SO_BATHY/restart2/mesh_mask.nc').rename({'x':'nav_lon', 'nav_lat_grid_T':'nav_lat'}).rename_dims({'nav_lon_grid_T':'x'}).to_netcdf('SO_mesh.nc')
#xr.load_dataset(path + 'SO_BATHY/restart2/domain_cfg_out.nc').rename({'x':'nav_lon', 'nav_lat_grid_T':'nav_lat'}).rename_dims({'nav_lon_grid_T':'x'}).to_netcdf('SO_domain.nc')

In [ ]:
domain = create_xgcm_domain(
    clean_coords( xr.load_dataset(path + 'GURVAN_5y/mesh.nc'), 2000),
    clean_coords(xr.load_dataset(path + 'GURVAN_5y/domain.nc'), 2000)
)

domain_big = create_xgcm_domain(
    clean_coords( xr.load_dataset(path + 'Scottia_low_150y/restart0/mesh_big.nc'), 2000   ),
    clean_coords( xr.load_dataset(path + 'Scottia_low_150y/restart0/domain_big.nc'), 2000 )
)

domain_free = create_xgcm_domain(
    clean_coords( xr.load_dataset(path + 'SO_BATHY/restart2/SO_mesh.nc'), 2000   ),
    clean_coords( xr.load_dataset(path + 'SO_BATHY/restart2/SO_domain.nc'), 2000 )
)

In [ ]:
domain_free

In [ ]:
Over_new    = (V_transport_on_z(domain, V_new).isel(time_counter=-2)).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6
Over_triad  = (V_transport_on_z(domain, V_triad).isel(time_counter=-2)).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6
Over_old    = (V_transport_on_z(domain, V_old).isel(time_counter=-2)).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6

# Meridional overturning streamfunction

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(20,15), sharex=True)
col1 = Over_new.plot(x='y_f', y='z', ax=axs[0,0], add_colorbar=False, cmap='RdBu_r')
cbar1 = fig.colorbar(col1, ax=axs[:,0], location='right', label='[Sv]')
Over_old.plot(x='y_f', y='z', ax=axs[1,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
Over_triad.plot(x='y_f', y='z', ax=axs[2,0], add_colorbar=False, vmin=cbar1.vmin, vmax=cbar1.vmax, cmap='RdBu_r')
cmap2 = (Over_new - Over_old).plot(x='y_f', y='z', ax=axs[0,1], add_colorbar=False, cmap='RdBu_r')
cbar2 = fig.colorbar(cmap2, ax=axs[:,1], location='right', label='[Sv]')
(Over_new - Over_triad).plot(x='y_f', y='z', ax=axs[1,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
(Over_old - Over_triad).plot(x='y_f', y='z', ax=axs[2,1], add_colorbar=False, vmin=cbar2.vmin, vmax=cbar2.vmax, cmap='RdBu_r')
axs[0,0].set_title('Old', fontsize=15)
axs[1,0].set_title('New', fontsize=15)
axs[2,0].set_title('Triad', fontsize=20)
axs[0,1].set_title('New - Old', fontsize=20)
axs[1,1].set_title('New - Triad', fontsize=20)
axs[2,1].set_title('Old - Triad', fontsize=20)

# 300 year runs

In [ ]:
import gsw as gsw

In [ ]:
V_s = clean_coords(xr.load_dataset(path + '1_deg_GM/restart5/NEVERWORLD_grid_V.nc'), 1700)
U_s = clean_coords(xr.load_dataset(path + '1_deg_GM/restart5/NEVERWORLD_grid_U.nc'), 1700)
T_s = clean_coords(xr.load_dataset(path + '1_deg_GM/restart5/NEVERWORLD_grid_T.nc'), 1700)

# V_z = clean_coords(xr.load_dataset(path + 'Scottia_low_150y/restart2/NEVERWORLD_grid_V.nc'), 1700)
# U_z = clean_coords(xr.load_dataset(path + 'Scottia_low_150y/restart2/NEVERWORLD_grid_U.nc'), 1700)
# T_z = clean_coords(xr.load_dataset(path + 'Scottia_low_150y/restart2/NEVERWORLD_grid_T.nc'), 1700)

# V_triad = combine_coords([V_200])
# U_triad = combine_coords([U_200])
# T_triad = combine_coords([T_200])

In [ ]:
V_z = clean_coords(xr.load_dataset(path + '1_deg_GM/restart4/NEVERWORLD_grid_V.nc'), 1700)
U_z = clean_coords(xr.load_dataset(path + '1_deg_GM/restart4/NEVERWORLD_grid_U.nc'), 1700)
T_z = clean_coords(xr.load_dataset(path + '1_deg_GM/restart4/NEVERWORLD_grid_T.nc'), 1700)



In [ ]:
T_mo_s = clean_coords(xr.load_dataset(path + '1_deg_GM/restart5/NEVERWORLD_grid_T_mo.nc'), 1700)
#T_mo_s = clean_coords(xr.load_dataset(path + 'nn_etau1/restart4/NEVERWORLD_grid_T_mo.nc'), 1700)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(20,10))
T_mo_s.mldr10_1max.isel(time_counter=-1).plot(ax=axs[0])
T_mo_z.mldr10_1max.isel(time_counter=-1).plot(ax=axs[1])
(T_mo_z.mldr10_1 - T_mo_s.mldr10_1).isel(time_counter=-1).plot(ax=axs[2])
axs[0].set_title('nn_etau1=1')
axs[1].set_title('nn_etau1=0')
axs[2].set_title('nn_etau1=0 - nn_etau1=1')
plt.tight_layout()

In [ ]:
T_s['sigma2'] = gsw.sigma2(T_s.soce, gsw.CT_from_pt(T_s.soce, T_s.toce))
#T_z['sigma0'] = gsw.sigma0(T_z.soce, gsw.CT_from_pt(T_z.soce, T_z.toce))

In [ ]:
V_s.y.isel(y=140)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
T_s.sigma0.isel(time_counter=-1, y=slice(1,-1), x=slice(1,-1), deptht=slice(20,-1)).mean('x').plot.contourf(levels=31)
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
T_z.sigma0.isel(time_counter=-1, y=slice(1,-1), x=slice(1,-1), deptht=slice(20,-1)).mean('x').plot.contourf(levels=31)
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
T_s.sigma0.isel(time_counter=-1, y=slice(1,-1), x=slice(25,-1), deptht=slice(0,-1)).mean('x').plot.contourf(levels=31)
ax.invert_yaxis()

In [ ]:
T_s.sigma0.isel(time_counter=-1, y=slice(1,-1), x=slice(1,-1), deptht=slice(0,-1)).mean('x').min()

In [ ]:
Over_s    = (V_transport_on_z(domain_free, V_s).isel(time_counter=-2)).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6

#Over_west    = (V_transport_on_z(domain_free, V_s).isel(time_counter=-2, x_c=slice(0,25))).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6
#Over_east    = (V_transport_on_z(domain_free, V_s).isel(time_counter=-2, x_c=slice(25,-1))).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6
#Over_z    = (V_transport_on_z(domain_free, V_z).isel(time_counter=-2)).sum(dim='x_c')[:,::-1].cumsum('depthw') / 1e6

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(20,10), sharey=True)
col = (-Over_west).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, add_colorbar=False, ax=axs[0])
cbar1 = fig.colorbar(col, ax=axs[:2], location='bottom', label='[Sv]')
(-Over_east).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r',vmin=cbar1.vmin, vmax=cbar1.vmax, levels=30, add_colorbar=False, ax=axs[1])
#(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[2])
axs[0].set_title('east')
axs[1].set_title(r'west')
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,10))
a = (-Over_east - Over_west).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs[0], label='[Sv]')
(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[1])
axs[1].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = (-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
#(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[2])
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = (-Over_z).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')

In [ ]:
Over_s.isel(depthw=slice(0,-1), y_f=126).min('depthw')

In [ ]:
vtra_den, e3v_den = V_transport_on_sigma(domain_free, V_s, T_s.sigma0)
Over_s_den    = vtra_den.isel(time_counter=-2).sum(dim='x_c')[:,::-1].cumsum('sigma_outer') / 1e6

#vtra_denz, e3v_denz = V_transport_on_sigma(domain_free, V_z, T_z.sigma0)
#Over_z_den    = vtra_denz.isel(time_counter=-2).sum(dim='x_c')[:,::-1].cumsum('sigma_outer') / 1e6

In [ ]:
vtra_den2, e3v_den2 = V_transport_on_sigma(domain_free, V_s, T_s.sigma2)
Over_s_den2    = vtra_den2.isel(time_counter=-2).sum(dim='x_c')[:,::-1].cumsum('sigma_outer') / 1e6

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = (-Over_s_den).plot.contourf(x='y_f', y='sigma_outer', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'')
#(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[2])
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = (-Over_s_den2).plot.contourf(x='y_f', y='sigma_outer', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{2}$ [ $kg$ / $m^3$ - 1000 ]')
#(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[2])
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = (-Over_z_den).plot.contourf(x='y_f', y='sigma_outer', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
#(Over_z-Over_s).isel(depthw=slice(0,-1)).plot.contourf(x='y_f', y='z', cmap='RdBu_r', levels=30, ax=axs[2])
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
ds_domain = domain_free.copy()
ds_domain['Over_s_den'] = Over_s_den

In [ ]:
grid_den = xgcm.Grid(ds_domain,
        coords={
            "Y": {"right": "y_f"},
            "Z": {"outer": "sigma_outer"}
        },
        periodic=False)

vtra_e3v = grid_den.transform(Over_s_den,
        'Z',
        domain_free.e3v.isel(y_f=100, x_c=40).data,
        #method='conservative',
        target_data=e3v_den.isel(x_c=40))

In [ ]:
vtra_e3v = vtra_e3v.assign_coords({'z': 5. - vtra_e3v.e3v.cumsum('e3v')})

In [ ]:
vtra_e3v.plot(x='y_f', y='z')

# Barotropic streamfunction

In [ ]:
#U_z['U'] = (U_z.uoce * U_z.e3u).sum('depthu')
U_s['U'] = (U_s.uoce * U_s.e3u).sum('depthu')

In [ ]:
#U_z_grid = domain_free.copy()
U_s_grid = domain_free.copy()

#U_z_grid['uoce']      = (['time_counter','z_c', 'y_c', 'x_f'], U_z.uoce.data)
U_s_grid['uoce']      = (['time_counter','z_c', 'y_c', 'x_f'], U_s.uoce.data)

#U_z_grid['e3u']      = (['time_counter','z_c', 'y_c', 'x_f'], U_z.e3u.data)
U_s_grid['e3u']      = (['time_counter','z_c', 'y_c', 'x_f'], U_s.e3u.data)

In [ ]:

metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3f'] # Z distances
}


grid_big = xgcm.Grid(domain_big,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

grid_free = xgcm.Grid(domain_free,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

In [ ]:
#U_z_grid['u_f'] = grid_free.interp(U_z_grid.uoce, 'Y')
U_s_grid['u_f'] = grid_free.interp(U_s_grid.uoce, 'Y')

In [ ]:
#U_z_grid['U'] = grid_free.integrate(U_z_grid.u_f, 'Z')
U_s_grid['U'] = grid_free.integrate(U_s_grid.u_f, 'Z')

In [ ]:
#U_z_grid['psi'] = (U_z_grid.U[:,::-1,:] * U_z_grid.e2f[::-1,:]).cumsum('y_f') / 1e6
U_s_grid['psi'] = (U_s_grid.U[:,::-1,:] * U_s_grid.e2f[::-1,:]).cumsum('y_f') / 1e6

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,15), sharey=True)
col = U_s_grid.isel(time_counter=-2).psi.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, ax=axs[1], add_colorbar=False)
cbar1 = fig.colorbar(col, ax=axs[:2], location='bottom', label='[Sv]')
U_z_grid.isel(time_counter=-2).psi.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r',vmin=cbar1.vmin, vmax=cbar1.vmax, levels=30, ax=axs[0], add_colorbar=False)
#(U_z_grid.psi - U_s_grid.psi).isel(time_counter=-2).plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, ax=axs[2])
axs[0].set_title(r'With Mid-Atlantic ridge in channel', fontsize=20)
axs[1].set_title(r'Without Mid-Atlantic ridge in channel', fontsize=20)
axs[0].set_ylabel('latitude [°N]')
axs[1].set_ylabel(' ')
axs[0].set_ylabel('longitude [°E]')
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
# U_s_grid = U_s_grid.assign_coords({'x_f': U_s_grid.x_f.values - 60.})
fig, axs = plt.subplots(1,1,figsize=(15,10))
a = U_s_grid.isel(time_counter=-2).psi.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
#(U_z_grid.psi - U_s_grid.psi).isel(time_counter=-2).plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, ax=axs[2])
axs.set_ylabel('latitude [°N]')
axs.set_xlabel('longitude [°E]')
#axs[2].set_title(r'z-coords - $\sigma$-coords')

In [ ]:
!jupyter nbconvert --to html ./Gurvan.ipynb --HTMLExporter.theme=dark 

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,7))
(U_s.uocetr_eff.isel(x=-1).sum(['depthu','y']) / 1e6).plot(label='old')
#(U_z.uocetr_eff.isel(x=-1).sum(['depthu','y']) / 1e6).plot(label='new')
ax.set_title('ACC transport over the last 50 years')
ax.set_ylabel('ACC transport [Sv]')
plt.legend()

# Test Forcings

In [ ]:
T_f = clean_coords(xr.load_dataset(path + 'SO_BATHY/restart1/NEVERWORLD_grid_T_mo.nc'), 2000)

T_f_new = clean_coords(xr.load_dataset(path + '1_deg_GM/restart1/NEVERWORLD_grid_T_mo.nc'), 2000)
U_f = clean_coords(xr.load_dataset(path + 'Test_forcings/NEVERWORLD_grid_U.nc'), 2000)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(25,15))
T_f.saltflx.isel(time_counter=0, x=10, y=slice(1,-1)).plot(ax=axs[0,1])
T_f.qns.isel(time_counter=0, x=10, y=slice(1,-1)).plot(ax=axs[0,0])
T_f.qsr.isel(time_counter=0, x=10, y=slice(1,-1)).plot(ax=axs[1,0])
U_f.utau.isel(time_counter=0, x=10).plot(ax=axs[1,1])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))
T_f.qns.isel(time_counter=6, x=10, y=slice(1,-1)).plot(ax=ax, label='old')
T_f_new.qns.isel(time_counter=6, x=10, y=slice(1,-1)).plot(ax=ax, label='new')
plt.legend()

In [ ]:
T_f

In [ ]:
orca_dom = xr.load_dataset(path + 'bathymetry/eORCA_R1_bathy_meter_v2.2.nc')

In [ ]:
orca_dom.Bathymetry.isel(y=slice(80,120), x=slice(260,270)).plot()

In [ ]:
orca_dom.Bathymetry.isel(y=230, x=slice(232,257)).plot()

In [ ]:
steps = np.hstack([
    orca_dom.Bathymetry.isel(y=230, x=slice(210,221)).values,
    orca_dom.Bathymetry.isel(y=230, x=slice(230,258)).values,
    orca_dom.Bathymetry.isel(y=230, x=slice(262,273)).values,
])

In [ ]:
steps.shape

In [ ]:
U_s['steps'] = (['x'], steps)

In [ ]:
plt.step(np.arange(len(steps)), steps, where='mid')

In [ ]:
steps.shape

In [ ]:
orca_dom.isel(y=235, x=slice(200,300))

In [ ]:
import cartopy.crs as ccrs

# Example - your x and y coordinates are in a Lambert Conformal projection
data_crs = ccrs.LambertConformal(central_longitude=-100)

# Transform the point - src_crs is always Plate Carree for lat/lon grid
x, y = data_crs.transform_point(-50, 26.0, src_crs=ccrs.PlateCarree())

# Now you can select data
orca_dom.Bathymetry.sel(y=y)

In [ ]:
U_s = clean_coords(xr.load_dataset(path + 'SO_BATHY/NEVERWORLD_grid_U.nc'), 1700)

In [ ]:
U_s.z.isel(time_counter=-1, depthu=-1, y=slice(10,70)).plot()

In [ ]:
5 * 24 * 360 * 3600. / 900 * 720


In [ ]:
5 * 720 / 900